In [1]:
!pip install -q -U transformers==4.37.2
!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q gradio
!pip install -q gTTS

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.4 MB/s eta 0:00:00

In [2]:
import torch
from transformers import BitsAndBytesConfig, pipeline
import whisper
import gradio as gr
import numpy as np
import re
import os
from PIL import Image
from gtts import gTTS

# Define quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Initialize model pipeline
model_id = "llava-hf/llava-1.5-7b-hf"
pipe = pipeline("image-to-text",
                model=model_id,
                model_kwargs={"quantization_config": quantization_config})

# Load Whisper model
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("medium", device=DEVICE)

# Check model details
print(f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
      f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

100%|██████████████████████████████████████| 1.42G/1.42G [00:15<00:00, 100MiB/s]


Model is multilingual and has 762,321,920 parameters.


In [3]:
import datetime
import warnings
from PIL import Image
import re
from gtts import gTTS
import whisper

# Logger file
def writehistory(text):
    tstamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    logfile = f'{tstamp}_log.txt'
    with open(logfile, 'a', encoding='utf-8') as f:
        f.write(text + '\n')

def img2txt(input_text, input_image):
    image = Image.open(input_image)
    writehistory(f"Input text: {input_text}")

    # Generate prompt based on input_text validity
    if isinstance(input_text, str) and input_text:
        prompt_instructions = f"Describe the image in detail based on the following prompt:\n{input_text}"
    else:
        prompt_instructions = ("Describe the image in detail. "
                               "Is it a painting or a photograph? "
                               "What are the predominant colors? "
                               "What does the image depict?")

    prompt = f"USER: <image>\n{prompt_instructions}\nASSISTANT:"
    outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 200})

    # Extract the response text
    if outputs and outputs[0]["generated_text"]:
        match = re.search(r'ASSISTANT:\s*(.*)', outputs[0]["generated_text"])
        return match.group(1) if match else "No response found."
    return "No response generated."

# Transcribes audio using Whisper model
def transcribe(audio):
    if not audio:
        return ''

    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    result = whisper.decode(model, mel, whisper.DecodingOptions())
    return result.text

# Converts text to speech and saves it as an audio file
def text_to_speech(text, file_path):
    language = 'en'
    audioobj = gTTS(text=text, lang=language, slow=False)
    audioobj.save(file_path)
    return file_path

In [4]:
def process_inputs(audio_path, image_path):
    speech_to_text_output = transcribe(audio_path)
    chatgpt_output = img2txt(speech_to_text_output, image_path) if image_path else "No image provided."
    processed_audio_path = text_to_speech(chatgpt_output, "Temp3.mp3")
    return speech_to_text_output, chatgpt_output, processed_audio_path

In [ ]:
iface = gr.Interface(
    fn=process_inputs,
    inputs=[
        gr.Audio(type="filepath", label="Voice Input (Microphone)"),
        gr.Image(type="filepath", label="Upload an Image")
    ],
    outputs=[
        gr.Textbox(label="Transcribed Text", lines=5, placeholder="Your transcribed text will appear here..."),
        gr.Textbox(label="AI Image Description", lines=5, placeholder="The AI's description of the image will appear here..."),
        gr.Audio(label="Audio Response")
    ],
    title="Image & Voice Interaction with OpenAI Whisper and Llava",
    description=(
        "This application allows you to interact with an AI by uploading an image and providing voice input. "
        "The AI will describe the image and respond to your voice input. "
        "Get started by uploading an image and speaking into your microphone."
    ),
    article=(
        "<p><b>Instructions:</b></p>"
        "<ul>"
        "<li><b>Step 1:</b> Upload an image that you want the AI to analyze.</li>"
        "<li><b>Step 2:</b> Speak into your microphone to provide additional input or ask questions about the image.</li>"
        "<li><b>Step 3:</b> View the AI's text-based description and listen to the audio response.</li>"
        "</ul>"
        "<p>This interface leverages OpenAI Whisper for speech-to-text transcription and Llava for image description generation.</p>"
    ),
    live=True  # Allow live updates for a more interactive experience
)

iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://30132eac4864dc4a92.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
